<a href="https://colab.research.google.com/github/carolineoliveira94/experimento/blob/main/text_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy pyLDAvis

In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import nltk
import re
from nltk import word_tokenize
from nltk.util import ngrams
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

tratamento de pontuação

In [ ]:
nltk.download('punkt')

conjunto de dados 

In [ ]:
news = pd.read_csv('https://www.dropbox.com/s/rvwjw8miow6jswy/articles.csv?dl=1')

In [ ]:
news.head()

In [ ]:
news['category'].head(50)

In [ ]:
news['subcategory'].head(60)

filtrar por turismo

In [ ]:
politica = news[news['category'] == 'poder']

In [ ]:
politica.info()

Limpar o texto

In [ ]:
def limpa_texto(texto):
  c = re.sub(r'\n', ' ', texto)
  c = re.sub(r'\r', ' ', c)

  c = c.replace(':', ' ')
  c = c.replace('/', ' ')
  c = re.sub(r'R\$', '', c)
  c = re.sub(r'W', ' ', c)

  c = re.sub(r'\s+', ' ', c)
  c = re.sub(r'^\s+', '', c)
  c = re.sub(r'\s+$', '', c)
  
  return c.lower()

In [ ]:
from spacy.lang.pt import Portuguese

In [ ]:
STOP_WORDS = spacy.lang.pt.stop_words.STOP_WORDS

In [ ]:
STOP_WORDS

In [ ]:
def remove_stop(texto):
  doc = nlp_pt(texto)
  return " ".join(x.text for x in doc if x.text not in STOP_WORDS)

In [ ]:
def limpa_texto2(texto):
  doc = nlp_pt(texto)
  return " ".join(x.lemma_ for x in doc if x.text not in STOP_WORDS)


análise exploratória 

1. numero de palavras no titulo das notícias 

In [ ]:
politica['title_n_words'] = politica['title'].apply(lambda x : len(x.split()))

In [ ]:
politica.head()

In [ ]:
politica.describe()

2. numero de caracteres no titulo das noticias 

In [ ]:
politica['title_n_chars'] = politica['title'].apply(lambda x : len(x.replace(" ", "")))

In [ ]:
politica.head()

In [ ]:
politica.describe()

In [ ]:
politica.title_n_words.hist();
plt.xlabel('número de palavras')
plt.ylabel('qtd')
plt.grid(b=None)
plt.show()

In [ ]:
politica.title_n_chars.hist();
plt.xlabel('número de caracteres')
plt.ylabel('qtd')
plt.grid(b=None)
plt.show()

limpeza dos dados 

In [ ]:
!spacy download es_core_news_sm


In [ ]:
nlp_pt = spacy.load('pt_core_news_sm')

In [ ]:
rev = politica.iloc[0]['title']

In [ ]:
rev

In [ ]:
rev = limpa_texto(rev)

In [ ]:
rev

In [ ]:
rev2 = politica.iloc[13]['title']

In [ ]:
rev2

In [ ]:
limpa_texto(rev2)

In [ ]:
rev = nlp_pt(rev)

In [ ]:
rev

In [ ]:
from ast import FormattedValue
def exibe_token(texto):
  col_names = ['ALPHA', 'PUNCT', 'LIKE NUM', 'POS']
  formatted_text = '{:>16}' * (len(col_names) + 1)
  print('\n', formatted_text.format('INPUT WORD', *col_names), '\n', '='*130)

  for token in texto:
    output = [token.text, token.is_alpha, token.is_punct, token.like_num, token.pos_]
    print(formatted_text.format(*output))


In [ ]:
exibe_token(rev)

In [ ]:
from spacy import displacy

In [ ]:
displacy.render(rev, style='dep', jupyter=True)

In [ ]:
rev2 = nlp_pt(rev2)

In [ ]:
exibe_token(rev2)

In [ ]:
politica['title_clean'] = politica['title'].apply(lambda x : limpa_texto(x))

In [ ]:
politica.head(10)

In [ ]:
politica['title_clean_no_stop'] = politica['title_clean'].apply(lambda x : remove_stop(x))

In [ ]:
politica.iloc[0]['title_clean']

In [ ]:
politica.iloc[0]['title_clean_no_stop']

lematização

In [ ]:
politica['title_lemmas'] = politica['title_clean_no_stop'].apply(lambda x : limpa_texto2(x))

In [ ]:
politica.iloc[0]['title_lemmas']

In [ ]:
politica

In [ ]:
politica.iloc[13]['title_lemmas']

*N*-grams (Quais tokens são mais comuns.)

In [ ]:
titulos = " ".join(politica['title_lemmas'])

In [ ]:
token = word_tokenize(titulos)

In [ ]:
bigrams = ngrams(token,2)

In [ ]:
print(Counter(bigrams).most_common(10))

In [ ]:
trigrams = ngrams(token,3)

In [ ]:
print(Counter(trigrams).most_common(10))

In [ ]:
vectorizer = CountVectorizer(min_df=10, max_df = 0.9, stop_words=STOP_WORDS, lowercase=True)

In [ ]:
data_vectorized = vectorizer.fit_transform(politica['title_lemmas'])

In [ ]:
NUM_TOPICS = 2 

In [ ]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=30, learning_method='online', verbose=True)

In [ ]:
data_lda = lda.fit_transform(data_vectorized)

In [ ]:
import pyLDAvis.sklearn

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')

In [ ]:
dash